# Residential Real Estate vs Supply, Inflation, and Interest Rate

## Description ##
Our goal is to determine how the value of residential real estate relates to supply, inflation, and interest rate.  Additionally, we will compare real estate returns to the returns of the  S&P 500 and XLRE (Real Estate Select Sector).

## Objectives ##
We intend to identify ideal periods to transact residential real estate based on valuation, inflation, and interest rate.

In [ ]:
# Imports
import pandas as pd
import os
from dotenv import load_dotenv
from fredapi import Fred
import hvplot.pandas
import yfinance as yf



In [ ]:
# Load Environment Variables
load_dotenv()

# setup fred api client
FRED_API_KEY = os.getenv('FRED_API_KEY')
fred = Fred(api_key=FRED_API_KEY)

## Load all Fred Data

In [ ]:
dataframes = {}
FRED_SERIES_KEYS = ["FPCPITOTLZGUSA", "FEDFUNDS", "MSACSR", "USSTHPI"]
for FRED_SERIES_KEY in FRED_SERIES_KEYS:
    series = fred.get_series(FRED_SERIES_KEY)
    frame = {
        "date": series.keys(),
        "value": series.values,
    }
    dataframes[FRED_SERIES_KEY] = pd.DataFrame(frame).set_index("date")
    dataframes[FRED_SERIES_KEY][f"pct_change {FRED_SERIES_KEY}"] = dataframes[FRED_SERIES_KEY].pct_change()
    dataframes[FRED_SERIES_KEY][f"diff {FRED_SERIES_KEY}"] = dataframes[FRED_SERIES_KEY]["value"].diff()
    dataframes[FRED_SERIES_KEY] = dataframes[FRED_SERIES_KEY].dropna()
    

## Residential Real Estate Value vs Inflation

In [ ]:
# Plot Inflation, consumer prices for the United States
display(dataframes["FPCPITOTLZGUSA"]["value"].hvplot(title="Inflation, consumer prices for the United States"))

# Plot House price Index for the United States
display(dataframes["USSTHPI"]["value"].hvplot(title="All Transactions House Price Index for the United States"))

In [ ]:
# Rename Columns to 'Inflation in %', Inflation pct_change'
dataframes["FPCPITOTLZGUSA"].columns = ['Inflation','Inflation pct_change','Difference in Inflation'] 
display(dataframes["FPCPITOTLZGUSA"].head())

# Rename Columns to 'House Price Index', HPI pct_change'
dataframes["USSTHPI"].columns = ['HPI','HPI pct_change','Difference in HPI']
dataframes["USSTHPI"].head()

#### Combine Infaltion and Housing Price Index for the United States

In [ ]:
# Use 'concat' function to combine the two DataFrames by matching indexes.
inflation_combined_df = pd.concat([dataframes["FPCPITOTLZGUSA"], dataframes["USSTHPI"]],axis="columns", join="inner")
inflation_combined_df.head()

In [ ]:
# Drop columns 'Inflation''Inflation pct_change','HPI and 'House Price Index'
inflation_combined_df = inflation_combined_df.drop(columns=["Inflation","Inflation pct_change","HPI","HPI pct_change"])
inflation_combined_df.head()

#### Conduct House Price Index and Inflation Analysis

In [ ]:
# Plot difference for combined dataframes
inflation_combined_df.hvplot(title="Housing Price Index, Inflation for the United States",xlabel="Date", ylabel="Difference")

## Residential Real Estate Value vs Federal Funds Effective Rate

In [ ]:
# Federal Effective Rate(Interest Rate)
display(dataframes["FEDFUNDS"]["value"].hvplot(title="Federal Effective Rate"))

# House price Index for the United States
display(dataframes["USSTHPI"]["HPI"].hvplot(title="All Transactions House Price Index for the United States"))

In [ ]:
# Rename columns to 'Interest Rate','Interest Rate pct_change','Difference in Interest Rate'
dataframes["FEDFUNDS"].columns=["Interest Rate","Interest Rate pct_change","Difference in Interest Rate"]
display(dataframes["FEDFUNDS"].head())

# Rename Columns to 'House Price Index', HPI pct_change'
dataframes["USSTHPI"].columns = ['HPI','HPI pct_change','Difference in HPI']
display(dataframes["USSTHPI"].head())

#### Combine Interest Rate and Housing Price Index for the United States

In [ ]:
# Use 'concat' function to combine the two DataFrames by matching indexes
fed_rate_combined_df = pd.concat([dataframes["FEDFUNDS"],dataframes["USSTHPI"]],axis="columns",join="inner")
fed_rate_combined_df.head()

In [ ]:
# Drop Columns 'Interest Rate','Interest Rate pct_change','HPI','HPI pct_change'
fed_rate_combined_df = fed_rate_combined_df.drop(columns=["Difference in Interest Rate","Interest Rate pct_change","HPI","HPI pct_change"])
fed_rate_combined_df.head()

#### Conduct House Price Index and Interest Rate Analysis

In [ ]:
# Plot difference for combined dataframes
fed_rate_combined_df.hvplot(title="House Price Index and Fedral Interest Rate", xlabel="Date",ylabel="Difference")

## Residential Real Estate Value vs Monthly Supply of New Houses

In [ ]:
# Plot Monthly Supply of New Houses in the United States
display(dataframes["MSACSR"]["value"].hvplot(title="Monthly Supply of New Houses in the United States"))

# Plot House price Index for the United States
display(dataframes["USSTHPI"]["HPI"].hvplot(title="All Transactions House Price Index for the United States"))

In [ ]:
# Rename Columns to 'Monthly Supply', 'MS pct_change','Difference in MS'
dataframes["MSACSR"].columns=["Monthly Supply","MS pct_change","Difference in MS"]
display(dataframes["MSACSR"].head())

# Rename Columns to 'House Price Index', HPI pct_change'
dataframes["USSTHPI"].columns = ['HPI','HPI pct_change','Difference in HPI']
dataframes["USSTHPI"].head()

#### Combine Monthly Suppky of New Houses and Housing Price Index for the United States

In [ ]:
# Use 'concat' function to combine the two DataFrames by matching indexes
monthly_supply_df = pd.concat([dataframes["MSACSR"],dataframes["USSTHPI"]],axis="columns",join="inner")
monthly_supply_df.head()

In [ ]:
# Drop columns 'Monthly Supply','MS pct_change','HPI','HPI pct_change'
monthly_supply_df = monthly_supply_df.drop(columns=["Difference in MS","MS pct_change","HPI","HPI pct_change"])
monthly_supply_df.head()

#### Conduct House Price Index and Monthly Supply of New Houses Analysis

In [ ]:
# Plot difference for combined dataframes
monthly_supply_df.hvplot(title="House Price Inde and Monthly Supply of New Houses in the United States", xlabel="Date(in Months)",ylabel="Difference")

## Residential Real Estate vs S&P500

In [ ]:
# Download the S&P500 Dataframe from the yfinance library

sp500_df = yf.download("^GSPC", interval="1mo", start="1985-01-01", end="2023-10-01")

# Get Closing cloumns and then calculate the monthly returns and drop NaN Values

sp500_close = sp500_df['Close']


sp500_returns = sp500_close.pct_change().dropna()

In [ ]:
# Plot the Monthly Returns

sp500_returns.hvplot(
    title='Monthly Returns of the S&P 500 Index',
    rot=90,
    ylabel='Rate of Return',
    xlabel='Date(in Months)'

)

## Residential Real Estate vs XLRE

In [ ]:
# Download the XLRE Fund Dataframe from the yfinance library

xlre_download = yf.download("XLRE", interval="1mo", start="1985-01-01", end="2023-10-01")

# Get Closing cloumns and then calculate the monthly returns and drop NaN Values

xlre_close = xlre_download['Close']


xlre_returns = xlre_close.pct_change().dropna()

In [ ]:
# Plot Monthly Returns 

xlre_returns.hvplot(
    title='Monthly Returns of The Real Estate Select Sector SPDR Fund (XLRE)',
    rot=90,
    ylabel='Rate of Return',
    xlabel='Date(in Months)'

)